Task 1.1

Create a simple model that analyzes text using an artificial neural network (ANN).

A small dataset of product reviews that have been labelled as negative (0) or positive (1) is provided in the Files - Exercises - Lab 1 folder, along with some code needed to extract information.

A suggested approach is first to try to train a network on the given data.
When that task has been concluded, improve the model performance by training with more data, using a dataset with a broader range of labels, using word embeddings to create unique sentence embeddings.


In [1]:
pip install gensim

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from matplotlib import pyplot
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from google.colab import files
uploaded = files.upload()

Saving amazon_cells_labelled.txt to amazon_cells_labelled (2).txt


In [7]:
def preprocess_pandas(data, columns):
    df_ = pd.DataFrame(columns=columns)
    data['Sentence'] = data['Sentence'].str.lower()
    data['Sentence'] = data['Sentence'].replace('[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+', '', regex=True)                      # remove emails
    data['Sentence'] = data['Sentence'].replace('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '', regex=True)    # remove IP address
    data['Sentence'] = data['Sentence'].str.replace('[^\w\s]','')                                                       # remove special characters
    data['Sentence'] = data['Sentence'].replace('\d', '', regex=True)                                                   # remove numbers
    for index, row in data.iterrows():
        word_tokens = word_tokenize(row['Sentence'])
        filtered_sent = [w for w in word_tokens if not w in stopwords.words('english')]
        df_.loc[len(df_)] = {
            "index": row['index'],
            "Class": row['Class'],
            "Sentence": " ".join(filtered_sent)
        }
    return data

# If this is the primary file that is executed (ie not an import of another file)
if __name__ == "__main__":
    # get data, pre-process and split
    data = pd.read_csv("amazon_cells_labelled.txt", delimiter='\t', header=None)
    data.columns = ['Sentence', 'Class']
    data['index'] = data.index                                          # add new column index
    columns = ['index', 'Class', 'Sentence']
    data = preprocess_pandas(data, columns)                             # pre-process
    training_data, validation_data, training_labels, validation_labels = train_test_split( # split the data into training, validation, and test splits
        data['Sentence'].values.astype('U'),
        data['Class'].values.astype('int32'),
        test_size=0.10,
        random_state=0,
        shuffle=True
    )

    # vectorize data using TFIDF and transform for PyTorch for scalability
    word_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,2), max_features=50000, max_df=0.5, use_idf=True, norm='l2')
    training_data = word_vectorizer.fit_transform(training_data)        # transform texts to sparse matrix
    training_data = training_data.todense()                             # convert to dense matrix for Pytorch
    vocab_size = len(word_vectorizer.vocabulary_)
    validation_data = word_vectorizer.transform(validation_data)
    validation_data = validation_data.todense()
    train_x_tensor = torch.from_numpy(np.array(training_data)).type(torch.FloatTensor)
    train_y_tensor = torch.from_numpy(np.array(training_labels)).long()
    validation_x_tensor = torch.from_numpy(np.array(validation_data)).type(torch.FloatTensor)
    validation_y_tensor = torch.from_numpy(np.array(validation_labels)).long()

In [5]:
train_loader = DataLoader(TensorDataset(train_x_tensor, train_y_tensor), batch_size=128, shuffle=True)
val_loader = DataLoader(TensorDataset(validation_x_tensor, validation_y_tensor), batch_size=128, shuffle=False) # No shuffle for validation, to ensure consistency of the validation

In [6]:
import copy
import matplotlib.pyplot as plt

# Define the model
network = nn.Sequential(
    nn.Linear(vocab_size, 128),
    nn.ReLU(),
    nn.Linear(128, 2)
)

optimizer = optim.Adam(network.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

epochs = 10
best_val_loss = float('inf')
best_model = None

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

# Training loop with accuracy
for epoch in range(epochs):
    network.train()
    running_train_loss = 0.0
    correct_train = 0
    total_train = 0

    for batch_x, batch_y in train_loader:
        prediction = network(batch_x)
        loss = loss_function(prediction, batch_y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_train_loss += loss.item()

        # Accuracy
        _, predicted = torch.max(prediction, 1)
        correct_train += (predicted == batch_y).sum().item()
        total_train += batch_y.size(0)

    avg_train_loss = running_train_loss / len(train_loader)
    train_accuracy = correct_train / total_train
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)

    # Validation
    network.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            prediction = network(batch_x)
            loss = loss_function(prediction, batch_y)
            running_val_loss += loss.item()

            _, predicted = torch.max(prediction, 1)
            correct_val += (predicted == batch_y).sum().item()
            total_val += batch_y.size(0)

    avg_val_loss = running_val_loss / len(val_loader)
    val_accuracy = correct_val / total_val
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)

    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model = copy.deepcopy(network)

    print(f"Epoch {epoch+1}/{epochs} - "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4f} - "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}")


Epoch 1/10 - Train Loss: 0.6921, Train Acc: 0.5033 - Val Loss: 0.6882, Val Acc: 0.4700
Epoch 2/10 - Train Loss: 0.6692, Train Acc: 0.5522 - Val Loss: 0.6709, Val Acc: 0.6600
Epoch 3/10 - Train Loss: 0.6310, Train Acc: 0.9667 - Val Loss: 0.6458, Val Acc: 0.7900
Epoch 4/10 - Train Loss: 0.5679, Train Acc: 0.9989 - Val Loss: 0.6149, Val Acc: 0.8400
Epoch 5/10 - Train Loss: 0.4979, Train Acc: 1.0000 - Val Loss: 0.5790, Val Acc: 0.8300
Epoch 6/10 - Train Loss: 0.4158, Train Acc: 1.0000 - Val Loss: 0.5389, Val Acc: 0.8500
Epoch 7/10 - Train Loss: 0.3386, Train Acc: 1.0000 - Val Loss: 0.5005, Val Acc: 0.8400
Epoch 8/10 - Train Loss: 0.2582, Train Acc: 1.0000 - Val Loss: 0.4667, Val Acc: 0.8500
Epoch 9/10 - Train Loss: 0.1918, Train Acc: 1.0000 - Val Loss: 0.4378, Val Acc: 0.8600
Epoch 10/10 - Train Loss: 0.1448, Train Acc: 1.0000 - Val Loss: 0.4160, Val Acc: 0.8700


Alright, we can see that the model is learning, but what can we do to improve the model?


We will retrain the network using the larger dataset of 25k items, as well as use word embeddings using the pre-trained vectors from Word2Vec.

Initially, we attempted to train our own vectors using Word2Vec, but we obtained similar validation loss/accuracy when compared to the previous model, so we opted to use pre-trained vectors instead.

In [3]:
from google.colab import files
uploaded = files.upload()

Saving amazon_cells_labelled_LARGE_25K.txt to amazon_cells_labelled_LARGE_25K (3).txt


In [8]:
# If this is the primary file that is executed (ie not an import of another file)
if __name__ == "__main__":
    # get data, pre-process and split
    data = pd.read_csv("amazon_cells_labelled_LARGE_25K.txt", delimiter='\t', header=None)
    data.columns = ['Sentence', 'Class']
    data['index'] = data.index                                          # add new column index
    columns = ['index', 'Class', 'Sentence']
    data = preprocess_pandas(data, columns)                             # pre-process
    training_data, validation_data, training_labels, validation_labels = train_test_split( # split the data into training, validation, and test splits
        data['Sentence'].values.astype('U'),
        data['Class'].values.astype('int32'),
        test_size=0.10,
        random_state=0,
        shuffle=True
    )


In [9]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import gensim.downloader as api

# Download Google's pre-trained Word2Vec model (100 billion words, 300D)
w2v_model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [10]:
sentences = [word_tokenize(s.lower()) for s in data['Sentence']] #tokenize sentence
import numpy as np

def sentence_to_vec(sentence, model, dim=300):
    tokens = word_tokenize(sentence.lower())
    vecs = [model[word] for word in tokens if word in model]
    if not vecs:
        return np.zeros(dim)
    return np.mean(vecs, axis=0)

X = np.array([sentence_to_vec(s, w2v_model) for s in data['Sentence']])
y = data['Class'].values

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
train_ds = TensorDataset(torch.tensor(X_train).float(), torch.tensor(y_train).long())
val_ds = TensorDataset(torch.tensor(X_val).float(), torch.tensor(y_val).long())

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=128, shuffle=False)

In [12]:
import copy
import matplotlib.pyplot as plt

# Define your network
network = nn.Sequential(
    nn.Linear(300, 128),
    nn.ReLU(),
    nn.Linear(128, 2)
)

optimizer = optim.Adam(network.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

epochs = 10
best_val_loss = float('inf')
best_model = None

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(epochs):
    network.train()
    running_train_loss = 0.0
    correct_train = 0
    total_train = 0

    for batch_x, batch_y in train_loader:
        prediction = network(batch_x)
        loss = loss_function(prediction, batch_y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_train_loss += loss.item()

        _, predicted = torch.max(prediction, 1)
        correct_train += (predicted == batch_y).sum().item()
        total_train += batch_y.size(0)

    avg_train_loss = running_train_loss / len(train_loader)
    train_accuracy = correct_train / total_train
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)

    # Validation
    network.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            prediction = network(batch_x)
            loss = loss_function(prediction, batch_y)
            running_val_loss += loss.item()

            _, predicted = torch.max(prediction, 1)
            correct_val += (predicted == batch_y).sum().item()
            total_val += batch_y.size(0)

    avg_val_loss = running_val_loss / len(val_loader)
    val_accuracy = correct_val / total_val
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)

    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model = copy.deepcopy(network)

    print(f"Epoch {epoch+1}/{epochs} - "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4f} - "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}")


Epoch 1/10 - Train Loss: 0.4820, Train Acc: 0.7581 - Val Loss: 0.3840, Val Acc: 0.8348
Epoch 2/10 - Train Loss: 0.3741, Train Acc: 0.8347 - Val Loss: 0.3592, Val Acc: 0.8416
Epoch 3/10 - Train Loss: 0.3610, Train Acc: 0.8406 - Val Loss: 0.3526, Val Acc: 0.8448
Epoch 4/10 - Train Loss: 0.3551, Train Acc: 0.8435 - Val Loss: 0.3499, Val Acc: 0.8392
Epoch 5/10 - Train Loss: 0.3510, Train Acc: 0.8437 - Val Loss: 0.3448, Val Acc: 0.8468
Epoch 6/10 - Train Loss: 0.3461, Train Acc: 0.8470 - Val Loss: 0.3425, Val Acc: 0.8432
Epoch 7/10 - Train Loss: 0.3420, Train Acc: 0.8477 - Val Loss: 0.3393, Val Acc: 0.8452
Epoch 8/10 - Train Loss: 0.3383, Train Acc: 0.8502 - Val Loss: 0.3368, Val Acc: 0.8436
Epoch 9/10 - Train Loss: 0.3348, Train Acc: 0.8524 - Val Loss: 0.3361, Val Acc: 0.8440
Epoch 10/10 - Train Loss: 0.3311, Train Acc: 0.8535 - Val Loss: 0.3365, Val Acc: 0.8484


Task 1.2

For this task, you will implement your transformer in PyTorch. You are instructed to follow this link: https://pytorch.org/hub/huggingface_pytorch-transformers/

## Task 1.2: Sentiment classification using transformer (BERT)

In [9]:
!pip install --no-deps --upgrade --no-cache-dir transformers torch tqdm
!pip install tqdm

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from matplotlib import pyplot
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### load and prepare dataset
We load the same dataset used in Task 1.1, split it into training and validation sets, and prepare it for the Transformer model.

In [21]:
from google.colab import files
uploaded = files.upload()

Saving amazon_cells_labelled_LARGE_25K.txt to amazon_cells_labelled_LARGE_25K (4).txt


In [22]:
def preprocess_pandas(data, columns):
    df_ = pd.DataFrame(columns=columns)
    data['Sentence'] = data['Sentence'].str.lower()
    data['Sentence'] = data['Sentence'].replace('[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+', '', regex=True)                      # remove emails
    data['Sentence'] = data['Sentence'].replace('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '', regex=True)    # remove IP address
    data['Sentence'] = data['Sentence'].str.replace('[^\w\s]','')                                                       # remove special characters
    data['Sentence'] = data['Sentence'].replace('\d', '', regex=True)                                                   # remove numbers
    for index, row in data.iterrows():
        word_tokens = word_tokenize(row['Sentence'])
        filtered_sent = [w for w in word_tokens if not w in stopwords.words('english')]
        df_.loc[len(df_)] = {
            "index": row['index'],
            "Class": row['Class'],
            "Sentence": " ".join(filtered_sent)
        }
    return data

# If this is the primary file that is executed (ie not an import of another file)
if __name__ == "__main__":
    # get data, pre-process and split
    data = pd.read_csv("amazon_cells_labelled.txt", delimiter='\t', header=None)
    data.columns = ['Sentence', 'Class']
    data['index'] = data.index                                          # add new column index
    columns = ['index', 'Class', 'Sentence']
    data = preprocess_pandas(data, columns)                             # pre-process
    training_data, validation_data, training_labels, validation_labels = train_test_split( # split the data into training, validation, and test splits
        data['Sentence'].values.astype('U'),
        data['Class'].values.astype('int32'),
        test_size=0.10,
        random_state=0,
        shuffle=True
    )

    # vectorize data using TFIDF and transform for PyTorch for scalability
    word_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,2), max_features=50000, max_df=0.5, use_idf=True, norm='l2')
    training_data = word_vectorizer.fit_transform(training_data)        # transform texts to sparse matrix
    training_data = training_data.todense()                             # convert to dense matrix for Pytorch
    vocab_size = len(word_vectorizer.vocabulary_)
    validation_data = word_vectorizer.transform(validation_data)
    validation_data = validation_data.todense()
    train_x_tensor = torch.from_numpy(np.array(training_data)).type(torch.FloatTensor)
    train_y_tensor = torch.from_numpy(np.array(training_labels)).long()
    validation_x_tensor = torch.from_numpy(np.array(validation_data)).type(torch.FloatTensor)
    validation_y_tensor = torch.from_numpy(np.array(validation_labels)).long()

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the Amazon review dataset LARGE 25k
data = pd.read_csv("amazon_cells_labelled_LARGE_25K.txt", delimiter='\t', header=None)
data.columns = ['Sentence', 'Class']

# Clean and split
data.dropna(inplace=True)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['Sentence'].tolist(),
    data['Class'].tolist(),
    test_size=0.1,
    random_state=42
)


### Tokenize


In [24]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors="pt")
val_encodings = tokenizer(val_texts, truncation=True, padding=True, return_tensors="pt")


### Create pytorch dataset

In [25]:
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)


### Load pretrained model


In [26]:
from transformers import BertForSequenceClassification
from torch.optim import AdamW

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Prepare detaset and train model


In [27]:
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


def evaluate_model(model, val_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch['labels'].cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    return acc

# Training with per-epoch evaluation
model.train()
for epoch in range(10):
    total_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")

    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    val_acc = evaluate_model(model, val_loader, device)
    print(f"Epoch {epoch+1} Completed: Loss = {total_loss:.4f}, Validation Accuracy = {val_acc:.4f}")


Epoch 1: 100%|██████████| 1407/1407 [06:20<00:00,  3.70it/s, loss=0.927]


Epoch 1 Completed: Loss = 322.9930, Validation Accuracy = 0.9268


Epoch 2: 100%|██████████| 1407/1407 [06:14<00:00,  3.76it/s, loss=0.00908]


Epoch 2 Completed: Loss = 184.9310, Validation Accuracy = 0.9252


Epoch 3: 100%|██████████| 1407/1407 [06:14<00:00,  3.76it/s, loss=0.00352]


...

Epoch 8 Completed: Loss = 40.4531, Validation Accuracy = 0.9236


Epoch 9: 100%|██████████| 1407/1407 [06:13<00:00,  3.76it/s, loss=0.000188]


Epoch 9 Completed: Loss = 30.4827, Validation Accuracy = 0.9200


Epoch 10: 100%|██████████| 1407/1407 [06:13<00:00,  3.76it/s, loss=0.000958]


Epoch 10 Completed: Loss = 43.6013, Validation Accuracy = 0.9124


Task 1.3

Here, you should compare of both models; you are requested to use the same test dataset for both ANN and the transformer to answer the following:

• Compare the performance of the two models and explain in which scenarios you would
prefer one over the other.

• How did the two models’ complexity, accuracy, and efficiency differ? Did one model
outperform the other in specific scenarios or tasks? If so, why?

• What insights did you obtain concerning data amount to train? Embedding utilized?
Architectural choices made?


## Task 1.3: Model Comparison

### ANN vs Transformer performance comparison

| Metric           | ANN Model         | Transformer (BERT)     |
|------------------|-------------------|-------------------------|
| Accuracy         | 84.84% (insert)   | 91.24% (insert)         |
| Precision/Recall | Lower on neutral texts | Higher semantic understanding |
| Training Speed   | Fast (Lightweight) | Slower (1h), resource intensive |
| Parameters       | Few (Simple network) | Millions (BERT)         |

---

### Insights

- **ANN Pros**:
  - Fast training, simpler to implement.
  - Works well on small, clean datasets.
  - Suitable for mobile or low-resource environments.

- **Transformer Pros**:
  - Better generalization on complex or nuanced texts.
  - Captures context and semantics thanks to attention mechanisms.
  - Scales well with large datasets.

- **Architecture & Embeddings**:
  - ANN used TF-IDF vectors or word embeddings (e.g. Word2Vec).
  - BERT used subword token embeddings + attention layers.

---

### Final Thoughts

If training data is limited or inference needs to be fast, an ANN is preferred. For real-world applications where context matters (e.g. chatbots, reviews, emotion detection), Transformer models significantly outperform traditional ANNs.